<a href="https://colab.research.google.com/github/1moi6/pyfuzzy-toolbox/blob/main/notebooks_colab/Aula_3/01_wang_mendel_didatico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Método de Wang-Mendel para Geração Automática de Regras Fuzzy

**Aula 3 - Minicurso de Sistemas de Inferência Fuzzy**

---

## 🎯 Objetivo

Neste notebook, vamos aplicar o **Método de Wang-Mendel (1992)** para **gerar automaticamente regras fuzzy** a partir de dados.

### Problema: Aproximar f(x) = sin(x) + 0.1x

- **Domínio**: x ∈ [0, 2π]
- **Função não-linear** (mais complexa que a reta!)
- **Objetivo**: Gerar regras fuzzy que aproximem a função

---

## 📖 Referência Original

**Wang, L. X., & Mendel, J. M. (1992)**. "Generating fuzzy rules by learning from examples."  
*IEEE Transactions on Systems, Man, and Cybernetics*, 22(6), 1414-1427.

- 🏆 **Mais de 5000 citações**
- 📊 Aplicações: Controle fuzzy, séries temporais, aproximação de funções
- 🧠 Algoritmo simples mas poderoso: **um único passo pelos dados**

---

## 🔧 Instalação

In [ ]:
!pip install pyfuzzy-toolbox[ml] -q

print('✅ pyfuzzy-toolbox instalado com sucesso!')

## 📦 Importações

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import fuzzy_systems as fs
from fuzzy_systems.learning import WangMendel

%matplotlib inline

# Configuração
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

print('✅ Bibliotecas importadas!')
print(f'   pyfuzzy-toolbox: {fs.__version__}')

## 🔍 Passo 1: O Problema - Aproximar f(x) = sin(x) + 0.1x

Vamos gerar dados de uma função **não-linear** e aplicar Wang-Mendel!

In [ ]:
# Função verdadeira
def funcao_real(x):
    """f(x) = sin(x) + 0.1x"""
    return np.sin(x) + 0.1 * x

# Gerar dados de treinamento
n_samples = 50
X_train = np.linspace(0, 2*np.pi, n_samples)
y_train = funcao_real(X_train) + np.random.normal(0, 0.05, n_samples)

# Dados de teste
X_test = np.linspace(0, 2*np.pi, 200)
y_true = funcao_real(X_test)

# Visualizar
plt.figure(figsize=(12, 6))
plt.plot(X_test, y_true, 'k-', linewidth=3, label='f(x) = sin(x) + 0.1x (Real)', alpha=0.7)
plt.scatter(X_train, y_train, color='red', s=100, zorder=5, 
           edgecolors='black', linewidths=1.5, label=f'Dados Observados (n={n_samples})', alpha=0.8)
plt.xlabel('x', fontsize=13)
plt.ylabel('y', fontsize=13)
plt.title('Problema: Aproximar função não-linear com regras fuzzy', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f'📊 Configuração:')
print(f'   • Função: f(x) = sin(x) + 0.1x')
print(f'   • Domínio: x ∈ [0, 2π] = [0, {2*np.pi:.3f}]')
print(f'   • Imagem: y ∈ [{y_train.min():.3f}, {y_train.max():.3f}]')
print(f'   • Amostras de treino: {n_samples}')
print(f'   • Amostras de teste: {len(X_test)}')

## 🧠 Passo 2: Criar e Treinar Modelo Wang-Mendel

Vamos criar um modelo com **15 partições** para capturar bem a não-linearidade!

### Os 5 Passos do Algoritmo Wang-Mendel:

1. **Particionar domínios**: Dividir entradas e saída em partições fuzzy
2. **Gerar regras**: Para cada amostra, criar uma regra
3. **Atribuir graus**: Calcular força de cada regra
4. **Resolver conflitos**: Se 2 regras têm mesma entrada mas saídas diferentes, manter a de maior grau
5. **Criar sistema fuzzy**: Construir FIS Mamdani com as regras

In [ ]:
# Criar modelo Wang-Mendel
wm = WangMendel(
    n_partitions=15,
    margin=0.10,
    input_names=['x'],
    output_name='y'
)

print('✅ Modelo Wang-Mendel criado!')
print('   • 15 partições (captura não-linearidades)')
print('   • Margem: 10% (evita overfitting)\n')

# Treinar (executa os 5 passos automaticamente)
wm.fit(X_train.reshape(-1, 1), y_train)

print('\n✅ Treinamento completo!')
print(f'   • Regras geradas: {wm.n_rules}')

## 📊 Passo 3: Visualizar Partições Fuzzy

Vamos ver como o Wang-Mendel dividiu o espaço de entrada e saída!

In [ ]:
# Visualizar partições da entrada
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Partições de x
x_universe = np.linspace(wm.fis.inputs['x'].universe[0], wm.fis.inputs['x'].universe[1], 300)
for term_name, term in wm.fis.inputs['x'].terms.items():
    mu = term.membership(x_universe)
    axes[0].plot(x_universe, mu, linewidth=2, label=term_name, alpha=0.7)

axes[0].set_title('Partições da Entrada x (15 termos linguísticos)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('Grau de Pertinência', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.05, 1.05])

# Partições de y
y_universe = np.linspace(wm.fis.outputs['y'].universe[0], wm.fis.outputs['y'].universe[1], 300)
for term_name, term in wm.fis.outputs['y'].terms.items():
    mu = term.membership(y_universe)
    axes[1].plot(y_universe, mu, linewidth=2, label=term_name, alpha=0.7)

axes[1].set_title('Partições da Saída y (15 termos linguísticos)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('y', fontsize=12)
axes[1].set_ylabel('Grau de Pertinência', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([-0.05, 1.05])

plt.tight_layout()
plt.show()

print('✅ Partições visualizadas!')

## 📜 Passo 4: Inspecionar Regras Geradas

Vamos ver algumas das regras que o Wang-Mendel criou automaticamente!

In [ ]:
print('='*70)
print('📜 BASE DE REGRAS GERADAS AUTOMATICAMENTE')
print('='*70)
print(f'\nTotal de regras: {len(wm.fis.rules)}\n')

print('Exemplos de regras (mostrando primeiras 10):\n')
for i, rule in enumerate(wm.fis.rules[:10], 1):
    # Extrair antecedentes
    ant_str = ', '.join([f'{var} é {term}' for var, term in rule.antecedents.items()])
    # Extrair consequentes
    cons_str = ', '.join([f'{var} é {term}' for var, term in rule.consequent.items()])
    print(f'   {i:2d}. SE {ant_str} ENTÃO {cons_str}')

if len(wm.fis.rules) > 10:
    print(f'\n   ... (e mais {len(wm.fis.rules) - 10} regras)')

print('\n' + '='*70)
print('💡 Cada regra foi gerada automaticamente a partir dos dados!')
print('='*70)

## 🧪 Passo 5: Avaliar Desempenho

Vamos testar o sistema e calcular métricas de qualidade!

In [ ]:
# Fazer predições
y_pred = wm.predict(X_test.reshape(-1, 1))

# Calcular métricas
mse = np.mean((y_pred - y_true) ** 2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(y_pred - y_true))
r2 = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2))

print('='*70)
print('📊 MÉTRICAS DE DESEMPENHO')
print('='*70)
print(f'   • MSE (Erro Médio Quadrático):      {mse:.6f}')
print(f'   • RMSE (Raiz do MSE):               {rmse:.6f}')
print(f'   • MAE (Erro Absoluto Médio):        {mae:.6f}')
print(f'   • R² (Coeficiente de Determinação): {r2:.6f}')
print('='*70)
print(f'\n✅ Qualidade da aproximação: {"Excelente!" if r2 > 0.95 else "Boa!" if r2 > 0.9 else "Razoável"}')

## 📈 Passo 6: Visualização da Aproximação

In [ ]:
# Gráfico principal
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Subplot 1: Aproximação
ax1.plot(X_test, y_true, 'k-', linewidth=3, label='Função Real', alpha=0.7)
ax1.scatter(X_train, y_train, color='red', s=80, zorder=5, 
           edgecolors='black', linewidths=1.5, label='Dados de Treino', alpha=0.8)
ax1.plot(X_test, y_pred, 'b-', linewidth=2.5, 
         label=f'Wang-Mendel (15 partições, {wm.n_rules} regras)', alpha=0.85)
ax1.set_xlabel('x', fontsize=13)
ax1.set_ylabel('y', fontsize=13)
ax1.set_title(f'Aproximação com Wang-Mendel | MSE = {mse:.6f} | R² = {r2:.4f}', 
             fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Subplot 2: Análise do erro
erro = y_pred - y_true
ax2.plot(X_test, erro, 'r-', linewidth=2.5, label='Erro')
ax2.fill_between(X_test, 0, erro, alpha=0.3, color='red')
ax2.axhline(y=0, color='black', linestyle='--', linewidth=1.5, alpha=0.5)
ax2.set_xlabel('x', fontsize=13)
ax2.set_ylabel('Erro (y_pred - y_true)', fontsize=13)
ax2.set_title(f'Análise do Erro | MAE = {mae:.6f}', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ Aproximação visualizada!')

## 🔍 Passo 7: Exemplo de Inferência Detalhada

Vamos analisar **passo a passo** como o sistema faz uma predição!

In [ ]:
# Escolher um ponto interessante (máximo da função seno)
x_teste = np.pi / 2
y_real = funcao_real(x_teste)

print('='*70)
print(f'🔍 ANÁLISE DETALHADA PARA x = π/2 ≈ {x_teste:.4f}')
print('='*70)
print(f'📌 Valor real: y = sin({x_teste:.4f}) + 0.1({x_teste:.4f}) = {y_real:.4f}')
print('='*70)
print()

# Fuzzificação
graus_x = wm.fis.inputs['x'].fuzzify(x_teste)
print('[ETAPA 1] FUZZIFICAÇÃO')
print('-'*70)
print(f'x = {x_teste:.4f}')
for termo, mu in graus_x.items():
    if mu > 0:
        print(f'  μ({termo}) = {mu:.3f}')

# Fazer predição
y_predito = wm.predict([[x_teste]])[0]

print()
print('='*70)
print(f'✅ Resultado: y_predito = {y_predito:.4f}')
print(f'✅ Valor real: y_real = {y_real:.4f}')
print(f'✅ Erro: |{y_predito:.4f} - {y_real:.4f}| = {abs(y_predito - y_real):.4f}')
print('='*70)

---

## 🎓 Conclusões

### ✅ O que aprendemos?

1. **Wang-Mendel é um Aproximador Universal**
   - Pode aproximar qualquer função contínua (teorema provado no paper original)
   - Funciona tanto para funções lineares quanto não-lineares
   - Não é interpolação! É aproximação robusta a ruído

2. **Geração Automática de Regras**
   - Regras são extraídas **automaticamente** dos dados
   - Algoritmo simples: **um único passo** pelos dados
   - Conflitos resolvidos mantendo regra com maior grau

3. **Interpretabilidade**
   - Cada regra é **linguística** e **interpretável**
   - Diferente de redes neurais (caixa-preta)
   - Podemos entender **como** o sistema funciona

4. **Parâmetros Importantes**
   - **n_partitions**: Afeta precisão e complexidade
   - **margin**: Importante para generalização (dados não vistos)
   - Trade-off: mais partições = menor erro, mas mais regras

---

### 🔬 Vantagens do Wang-Mendel

✅ Rápido (um único passo pelos dados)  
✅ Regras interpretáveis  
✅ Não requer especialista  
✅ Funciona para classificação e regressão  
✅ Robusto a ruído  

### ⚠️ Limitações

⚠️ Número de partições fixo (escolhido a priori)  
⚠️ Pode gerar muitas regras com muitas entradas (explosão combinatória)  
⚠️ Não refina as regras (diferente do ANFIS que vem na Aula 4!)  

---

### 🚀 Próximos Passos

1. **Regressão Linear**: Aplicar Wang-Mendel a uma reta (`02_wang_mendel_reta.ipynb`)
2. **Classificação**: Aplicar Wang-Mendel ao dataset Iris (`03_wang_mendel_iris.ipynb`)
3. **ANFIS** (Aula 4): Usar Wang-Mendel para inicializar + refinar com backpropagation

---

### 📚 Recursos

- **PyPI**: https://pypi.org/project/pyfuzzy-toolbox/
- **GitHub**: https://github.com/1moi6/pyfuzzy-toolbox

### 📖 Referências

1. **Wang, L. X., & Mendel, J. M. (1992)**. "Generating fuzzy rules by learning from examples."  
   *IEEE Transactions on Systems, Man, and Cybernetics*, 22(6), 1414-1427.

2. **Ross, T. J. (2010)**. *Fuzzy Logic with Engineering Applications*. 3rd ed. Wiley.

3. **Jang et al. (1997)**. *Neuro-Fuzzy and Soft Computing*. Prentice Hall.

---

*Notebook desenvolvido para o Minicurso de Sistemas de Inferência Fuzzy - Aula 3 - 2025*  
*Usando pyfuzzy-toolbox v1.0.0*